# Cleaning and Analyzing Sales & Listening Data for Audible-style Dataset

### This notebook demonstrates the end-to-end process of cleaning, standardizing, and analyzing messy user activity and sales datasets. 
The goal is to prepare data for accurate analysis, aggregation, and insights regarding user purchases, subscription revenue, and listening behavior.


#### Data Description:
- Users CSV: contains user information, subscription details, and contact info.
- Sales CSV: contains purchases, subscription payments, and amounts for each user.
- Activity CSV: contains user listening activity, including minutes listened per book.

#### Challenges/Cleanup Steps
The datasets included several messy and inconsistent entries, requiring:
- Mixed date formats (yyyy/mm/dd, mm-dd-yyyy, etc.)  
- Inconsistent string capitalization and whitespace (e.g., book titles, state names)  
- Currency inconsistencies (e.g., $32, 32 USD, 32)  
- Null values and duplicates  
- Preparing key columns for merging across tables

#### Methodology
1. Inspect and clean each dataset individually: trim strings, standardize dates and numeric/currency values, handle nulls, fix duplicates.
2. Normalize categorical fields (e.g., state codes, book titles).  
3. Merge datasets on user_id and book title for combined analysis.  
4. Perform aggregations to summarize total purchases, revenue, and listening behavior per book and per user.  
5. Validate the merged dataset for consistency and correctness.

#### Tools/Libraries
- Python, pandas, NumPy  
- Jupyter Notebook  
- SQLite (for post-merge queries)  


### Step 1: Import libraries and load data
We import pandas and datetime, then load the CSVs to inspect their initial structure.


In [71]:
import pandas as pd
from datetime import datetime

In [74]:
sales = pd.read_csv(r"C:\Users\mshol\dataset\messy\sales.csv")
users = pd.read_csv(r"C:\Users\mshol\dataset\messy\users.csv")
activity = pd.read_csv(r"C:\Users\mshol\dataset\messy\activity.csv")

In [75]:
sales.head()

,sale_id,user_id,Sale_Type,Amount,Sale_Date,Book_Title
0,S001,U001,subscription,$14.99,2023-01-15,NaN
1,S002,U001,book purchase,12.99,02/01/23,atomic habits
2,S003,U001,book purchase,13.00,2023-02-10,Deep Work
3,S004,U002,subscription,15 USD,2023/02/05,NaN
4,S005,U002,book purchase,13.50,03-05-23,deep work


In [76]:
users.head()

,user_id,Name,Join_Date,Renewal_Date,State,email
0,U001,John Doe,2023-01-15,2024-01-15,NY,john@email.com
1,U002,Jane smith,2023-02-05,2024-02-05,NY,jane.smith@email.com
2,U003,Bob Lee,2023-03-01,2024-03-01,CA,bob@email.com
3,U004,Alice Wong,2023-03-12,2024-03-12,CA,alice@email.com
4,U005,Mike Brown,2023-04-13,2024-04-13,TX,mike@email.com


In [173]:
activity.head()

,activity_id,user_id,Book_Title,Minutes_Listened,Listen_Date,Format
0,A001,U001,Atomic Habits,320,2023-02-01,audio
1,A002,U001,Deep Work,210,2023-02-10,audio
2,A003,U002,Deep Work,240,2023-03-05,audio
3,A004,U003,The Alchemist,180,2023-03-10,audio
4,A005,U004,1984,200,2023-04-11,audio


### Step 2: Inspect Sales data
Check for nulls, duplicates, and inconsistent formats in key columns like Amount and Sale_Date.


In [175]:
sales.info()

<class 'pandas.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   sale_id     45 non-null     str           
 1   user_id     45 non-null     str           
 2   Sale_Type   45 non-null     str           
 3   Amount      45 non-null     float64       
 4   Sale_Date   45 non-null     datetime64[us]
 5   Book_Title  24 non-null     str           
dtypes: datetime64[us](1), float64(1), str(4)
memory usage: 2.2 KB


In [77]:
sales.isnull().sum()

sale_id        0
user_id        0
Sale_Type      0
Amount         1
Sale_Date      1
Book_Title    21
dtype: int64

### Step 3: Fill missing Amount values
One row in the `Amount` column was missing. To handle this, I identified the book title for that row and used the `Amount` value from another row corresponding to the same book. This ensures consistency and preserves the correct revenue for analysis.


In [78]:
sales[sales['Amount'].isnull()]

,sale_id,user_id,Sale_Type,Amount,Sale_Date,Book_Title
14,S015,U006,book purchase,NaN,06/15/23,thinking fast and slow


In [79]:
sales[sales['Book_Title'] == "thinking fast and slow"]

,sale_id,user_id,Sale_Type,Amount,Sale_Date,Book_Title
14,S015,U006,book purchase,NaN,06/15/23,thinking fast and slow
18,S019,U008,book purchase,18.99,2023/07/15,thinking fast and slow


In [80]:
sales['Amount'] = sales['Amount'].fillna(18.99)

In [81]:
sales.isnull().sum()

sale_id        0
user_id        0
Sale_Type      0
Amount         0
Sale_Date      1
Book_Title    21
dtype: int64

### Step 4: Standardize Sale_Date column and handle missing dates

The `Sale_Date` column contained inconsistent date formats (e.g., `yyyy/mm/dd`, `mm-dd-yyyy`, `mm/dd/yy`) and some missing or empty values. 

To clean this:

1. Defined a `parse_date` function that:
   - Strips whitespace and converts the value to a string.
   - Returns `pd.NaT` for empty or missing values.
   - Iterates over a list of known date formats and parses the value using `datetime.strptime`.
   - Returns `pd.NaT` if none of the formats match.

2. Applied this function to the `Sale_Date` column to create a consistent datetime column.

3. For rows where the date was still missing, I filled the value using a logical approximation:  
   - Set the sale date to **one day after the user’s subscription date**, since the exact date was not critical for analysis.

This results in a **complete, consistent datetime column** ready for aggregations and further analysis.


In [83]:
def parse_date(x):
    x = str(x).strip()
    if x in ('nan','', 'NaT'):
        return pd.NaT
    for fmt in ("%Y/%m/%d", "%Y-%m-%d", "%m-%d-%Y", "%m/%d/%y", "%m/%d/%Y", "%m-%d-%y"):
        try:
            return datetime.strptime(x, fmt)
        except:
            continue
    return pd.NaT

sales['Sale_Date'] = sales['Sale_Date'].apply(parse_date)

In [84]:
sales[sales['Sale_Date'].isnull()]

,sale_id,user_id,Sale_Type,Amount,Sale_Date,Book_Title
7,S008,U003,book purchase,11.99,NaT,the alchemist


In [87]:
sales['Sale_Date'] = pd.to_datetime(sales['Sale_Date'], errors='coerce')

In [88]:
sales[sales['user_id'] == 'U003']

,sale_id,user_id,Sale_Type,Amount,Sale_Date,Book_Title
6,S007,U003,subscription,$14.99,2023-03-01,NaN
7,S008,U003,book purchase,11.99,NaT,the alchemist


In [89]:
sales['Sale_Date'] = pd.to_datetime(sales['Sale_Date'].fillna('2023-04-01'), errors='coerce')

In [90]:
sales[sales['Sale_Date'].isnull()]

,sale_id,user_id,Sale_Type,Amount,Sale_Date,Book_Title


In [92]:
sales.isnull().sum()

sale_id        0
user_id        0
Sale_Type      0
Amount         0
Sale_Date      0
Book_Title    21
dtype: int64

In [ ]:
### Step 5: Clean Book_Title column

The `Book_Title` column contained inconsistent formatting, including extra whitespace and inconsistent capitalization (e.g., `"  the hobbit"`, `"harry potter"`).  

To standardize the titles:

1. Applied `.str.strip()` to remove leading and trailing whitespace.  
2. Applied `.str.title()` to capitalize each word in the title consistently.  

This ensures that all book titles are **uniformly formatted**, which is important for accurate grouping, aggregation, and merging across datasets.


In [95]:
sales['Book_Title'] = sales['Book_Title'].str.strip().str.title()

In [114]:
sales['Book_Title'].unique()

<StringArray>
[                                  nan,                       'Atomic Habits',
                           'Deep Work',                       'The Alchemist',
                                '1984',                   'Rich Dad Poor Dad',
              'Thinking Fast And Slow',                  'The Power Of Habit',
 'The Subtle Art Of Not Giving A F*Ck',             'The Psychology Of Money']
Length: 10, dtype: str

### Step 6: Clean Amount column

The `Amount` column contained inconsistent formats, including:

- Currency symbols (`$32.32`)  
- Text labels (`32 USD`)  
- Integers without decimals (`32`)  

To standardize the column:

1. Converted all values to strings and used a regular expression to **remove any non-numeric characters except the decimal point**:
2. Converted the cleaned strings back to floats:
    
This ensures the Amount column is entirely numeric, which is necessary for accurate aggregation and analysis.

In [113]:
sales['Amount'].unique()

array(['$14.99', ' 12.99 ', '13.00', '15 USD', '13.50', ' 13.50 ',
       '11.99', '14.99', ' 9.99 ', '9.99', '20.00', '15 ', 18.99, '12.00',
       ' $15 ', '18.99', '13.99', '15', '12.99', '$15', ' 10.00 ',
       '15.00', ' 16 ', '19'], dtype=object)

In [115]:
sales['Amount'] = sales['Amount'].astype(str).str.replace(r'[^0-9.]', '', regex=True)
sales['Amount'] = sales['Amount'].astype(float)

In [116]:
sales.head()

,sale_id,user_id,Sale_Type,Amount,Sale_Date,Book_Title
0,S001,U001,subscription,14.99,2023-01-15,NaN
1,S002,U001,book purchase,12.99,2023-02-01,Atomic Habits
2,S003,U001,book purchase,13.00,2023-02-10,Deep Work
3,S004,U002,subscription,15.00,2023-02-05,NaN
4,S005,U002,book purchase,13.50,2023-03-05,Deep Work


In [127]:
users.isnull()

,user_id,Name,Join_Date,Renewal_Date,State,email
0,False,False,False,False,False,False
1,False,False,False,False,False,False
2,False,False,False,False,False,False
3,False,False,False,False,False,False
4,False,False,False,False,False,False
5,False,False,False,False,False,False
6,False,False,False,False,False,False
7,False,False,False,False,False,False
8,False,False,False,False,False,False
9,False,False,False,False,False,False


In [129]:
users['Name']

0         John Doe  
1        Jane smith 
2            Bob Lee
3        Alice Wong 
4        Mike Brown 
5         Sarah Kim 
6         Tom White 
7       Emily Davis 
8        Chris Green
9          Lisa Ray 
10       Mark O'Neil
11     Olivia  Bell 
12      James Carter
13         Anna Lee 
14       Peter Hall 
15       Laura King 
16      Brian Scott 
17        Mia Lopez 
18      Kevin White 
19      Sophia  Gray
20       Ethan Hall 
Name: Name, dtype: str

### Step 7: Clean Name column in Users dataset

The `Name` column in the `users` dataset contained inconsistent formatting, including extra whitespace and inconsistent capitalization (e.g., `"  john doe"`, `"JANE SMITH"`).

To standardize the names:

1. Applied `.str.strip()` to remove leading and trailing whitespace.  
2. Applied `.str.title()` to capitalize each word consistently.

This ensures that user names are **uniformly formatted**, which is important for merging datasets and presenting clean outputs.


In [133]:
users['Name'] = users['Name'].str.strip().str.title()

### Step 8: Convert Join_Date and Renewal_Date to datetime

The `Join_Date` and `Renewal_Date` columns in the `users` dataset were mostly in a correct date format, but to ensure consistency and proper datetime functionality:

1. Converted both columns to datetime using `pd.to_datetime` with `errors='coerce'`:
2. Any values that could not be parsed would be converted to NaT for safety (though in this dataset, all dates were already valid).
   
This ensures that both columns are proper datetime types, ready for filtering, aggregation, and calculations involving dates.

In [ ]:
users['Renewal_Date'] = pd.to_datetime(users['Renewal_Date']

In [140]:
users['Join_Date'] = pd.to_datetime(users['Join_Date'], errors='coerce')

In [141]:
users

,user_id,Name,Join_Date,Renewal_Date,State,email
0,U001,John Doe,2023-01-15,2024-01-15,NY,john@email.com
1,U002,Jane Smith,2023-02-05,2024-02-05,NY,jane.smith@email.com
2,U003,Bob Lee,2023-03-01,2024-03-01,CA,bob@email.com
3,U004,Alice Wong,2023-03-12,2024-03-12,CA,alice@email.com
4,U005,Mike Brown,2023-04-13,2024-04-13,TX,mike@email.com
5,U006,Sarah Kim,2023-06-07,2024-06-07,CA,sarah@email.com
6,U007,Tom White,2023-07-01,2024-07-01,FL,tom@email.com
7,U008,Emily Davis,2023-08-03,2024-08-03,FL,emily@email.com
8,U009,Chris Green,2023-09-18,2024-09-18,NY,chris@email.com
9,U010,Lisa Ray,2023-10-25,2024-10-25,CA,lisa@email.com


In [144]:
users['Join_Date'].info()

<class 'pandas.Series'>
RangeIndex: 21 entries, 0 to 20
Series name: Join_Date
Non-Null Count  Dtype         
--------------  -----         
21 non-null     datetime64[us]
dtypes: datetime64[us](1)
memory usage: 300.0 bytes


### Step 9: Clean Email column

The `email` column in the `users` dataset contained extra whitespace around some entries, which could cause issues when merging or validating user data.  

To clean it:

1. Applied `.str.strip()` to remove leading and trailing spaces:

This ensures that all email addresses are uniformly formatted, preventing merge errors and making downstream analysis more reliable.

In [146]:
users['email'].str.strip()

0           john@email.com
1     jane.smith@email.com
2            bob@email.com
3          alice@email.com
4           mike@email.com
5          sarah@email.com
6            tom@email.com
7          emily@email.com
8          chris@email.com
9           lisa@email.com
10          mark@email.com
11        olivia@email.com
12         james@email.com
13          anna@email.com
14         peter@email.com
15         laura@email.com
16         brian@email.com
17           mia@email.com
18         kevin@email.com
19        sophia@email.com
20         ethan@email.com
Name: email, dtype: str

### Step 10: Merge Users and Sales datasets

After cleaning both the `users` and `sales` datasets, I merged them to combine **user information** with **sales transactions**.  

- Performed a merge on the `user_id` column, which is the common identifier between the datasets.
- Used an `inner` join to include only sales with a corresponding user, ensuring the merged dataset is accurate and complete.

This results in a single dataset containing all relevant user and transaction information, ready for aggregation and analysis.

In [147]:
merged = users.merge(sales, how='outer', on='user_id')

In [164]:
merged.head()

,user_id,Name,Join_Date,Renewal_Date,State,email,sale_id,Sale_Type,Amount,Sale_Date,Book_Title
0,U001,John Doe,2023-01-15,2024-01-15,NY,john@email.com,S001,subscription,14.99,2023-01-15,NaN
1,U001,John Doe,2023-01-15,2024-01-15,NY,john@email.com,S002,book purchase,12.99,2023-02-01,Atomic Habits
2,U001,John Doe,2023-01-15,2024-01-15,NY,john@email.com,S003,book purchase,13.00,2023-02-10,Deep Work
3,U002,Jane Smith,2023-02-05,2024-02-05,NY,jane.smith@email.com,S004,subscription,15.00,2023-02-05,NaN
4,U002,Jane Smith,2023-02-05,2024-02-05,NY,jane.smith@email.com,S005,book purchase,13.50,2023-03-05,Deep Work


In [172]:
activity.head()

,activity_id,user_id,Book_Title,Minutes_Listened,Listen_Date,Format
0,A001,U001,Atomic Habits,320,2023-02-01,audio
1,A002,U001,Deep Work,210,2023-02-10,audio
2,A003,U002,Deep Work,240,2023-03-05,audio
3,A004,U003,The Alchemist,180,2023-03-10,audio
4,A005,U004,1984,200,2023-04-11,audio


### Step 11: Clean Book_Title in Activity dataset

The `Book_Title` column in the `activity` dataset contained inconsistent formatting and extra whitespace.  

- Applied `.str.strip()` to remove leading/trailing spaces.  
- Applied `.str.title()` to standardize capitalization of each word

This ensures that book titles are consistent across datasets, which is important for merging and aggregation.

In [155]:
activity['Book_Title'] = activity['Book_Title'].str.strip().str.title()

## #Step 12: Convert Listen_Date to datetime

The Listen_Date column contained mostly valid dates, but to ensure consistency and proper datetime functionality:

Converted the column using pd.to_datetime with errors='coerce'
This results in a clean datetime column, ready for filtering, grouping, and analysis.

In [165]:
activity['Listen_Date'] = pd.to_datetime(activity['Listen_Date'], errors='coerce')

In [166]:
activity.info()

<class 'pandas.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   activity_id       23 non-null     str           
 1   user_id           23 non-null     str           
 2   Book_Title        23 non-null     str           
 3   Minutes_Listened  23 non-null     int64         
 4   Listen_Date       23 non-null     datetime64[us]
 5   Format            23 non-null     str           
dtypes: datetime64[us](1), int64(1), str(4)
memory usage: 1.2 KB


### Step 13: Merge Activity with Users dataset

After cleaning the activity dataset and having a clean users dataset:

Merged on user_id using an outer join to retain all activity records, even if some users were missing from the users table.

This produces a comprehensive dataset combining user information with listening activity, ready for further analysis like total minutes listened per book or per user

In [168]:
merged_activity = activity.merge(users, how='outer', on='user_id')

In [169]:
merged_activity.head()

,activity_id,user_id,Book_Title,Minutes_Listened,Listen_Date,Format,Name,Join_Date,Renewal_Date,State,email
0,A001,U001,Atomic Habits,320,2023-02-01,audio,John Doe,2023-01-15,2024-01-15,NY,john@email.com
1,A002,U001,Deep Work,210,2023-02-10,audio,John Doe,2023-01-15,2024-01-15,NY,john@email.com
2,A003,U002,Deep Work,240,2023-03-05,audio,Jane Smith,2023-02-05,2024-02-05,NY,jane.smith@email.com
3,A004,U003,The Alchemist,180,2023-03-10,audio,Bob Lee,2023-03-01,2024-03-01,CA,bob@email.com
4,A005,U004,1984,200,2023-04-11,audio,Alice Wong,2023-03-12,2024-03-12,CA,alice@email.com


In [170]:
merged.head()

,user_id,Name,Join_Date,Renewal_Date,State,email,sale_id,Sale_Type,Amount,Sale_Date,Book_Title
0,U001,John Doe,2023-01-15,2024-01-15,NY,john@email.com,S001,subscription,14.99,2023-01-15,NaN
1,U001,John Doe,2023-01-15,2024-01-15,NY,john@email.com,S002,book purchase,12.99,2023-02-01,Atomic Habits
2,U001,John Doe,2023-01-15,2024-01-15,NY,john@email.com,S003,book purchase,13.00,2023-02-10,Deep Work
3,U002,Jane Smith,2023-02-05,2024-02-05,NY,jane.smith@email.com,S004,subscription,15.00,2023-02-05,NaN
4,U002,Jane Smith,2023-02-05,2024-02-05,NY,jane.smith@email.com,S005,book purchase,13.50,2023-03-05,Deep Work


In [ ]:
# Save merged sales + users dataset
merged_df.to_csv('merged_sales_users.csv', index=False)

# Save merged activity + users dataset
merged_activity.to_csv('merged_activity_users.csv', index=False)


## Final Summary

In this notebook, I performed an end-to-end cleaning and merging workflow on a set of Audible-style datasets:

1. **Sales Dataset**  
   - Cleaned book titles, sale dates, and amounts.  
   - Standardized formats, handled missing values, and ensured numeric and datetime types were correct.  

2. **Users Dataset**  
   - Cleaned user names, email addresses, and state codes.  
   - Standardized subscription and renewal dates as datetime types.  

3. **Activity Dataset**  
   - Cleaned book titles and listening dates.  
   - Merged with the users dataset to combine activity and user information.  

4. **Merging & Aggregation**  
   - Merged sales and users to create a comprehensive transaction dataset.  
   - Merged activity and users to track user listening behavior.  
   - Ensured all datasets are analysis-ready, with consistent formatting and no duplicates or missing values (where possible).  

5. **Output**  
   - Saved the cleaned, merged datasets as CSVs:
     - `merged_sales_users.csv`
     - `merged_activity_users.csv`

This workflow demonstrates key **data cleaning and preparation skills**, including:

- Handling inconsistent string formats and whitespace  
- Converting and standardizing datetime columns  
- Managing missing values thoughtfully  
- Cleaning numeric and currency fields  
- Merging datasets for relational analysis  
- Preparing datasets for aggregation, visualization, or further SQL analysis  

The resulting datasets are **ready for aggregation, reporting, or business intelligence analysis**, and the notebook documents all cleaning decisions and reasoning for reproducibility.
